### Import dependencies

In [1]:
import os
import numpy as np
import mne
from mne.utils import logger
from _tsv_handler import _from_tsv
from collections import defaultdict


In [1]:
from _electrode_reader import _handle_electrodes_reading

ImportError: No module named _digitization

### Check files

In [3]:
eeg_path = '/home/localadmin/Documents/research/2_projects/CMP_EEG/ds002718-cmpeeg/sub-002/eeg/'
for elem in os.listdir(eeg_path):
    print(elem)

sub-002_task-FaceRecognition_events.tsv
._sub-002_task-FaceRecognition_eeg.json
sub-002_task-FaceRecognition_electrodes.tsv
sub-002_task-FaceRecognition_eeg.json
sub-002_task-FaceRecognition_eeg.set
sub-002_task-FaceRecognition_coordsystem.json
sub-002_task-FaceRecognition_events.json
sub-002_task-FaceRecognition_eeg.fdt
sub-002_task-FaceRecognition_channels.tsv


### Create MNE raw object

In [ ]:
raw = mne.io.read_raw_eeglab(os.path.join(eeg_path,'sub-002_task-FaceRecognition_eeg.set'),uint16_codec='utf-8')

### Load channels

In [ ]:
def _handle_channels_reading(channels_fname, bids_fname, raw):
    """Read associated channels.tsv and populate raw.
    Updates status (bad) and types of channels.
    """
    logger.info('Reading channel info from {}.'.format(channels_fname))
    channels_dict = _from_tsv(channels_fname)

    # First, make sure that ordering of names in channels.tsv matches the
    # ordering of names in the raw data. The "name" column is mandatory in BIDS
    ch_names_raw = list(raw.ch_names)
    ch_names_tsv = channels_dict['name']
    if ch_names_raw != ch_names_tsv:

        msg = ('Channels do not correspond between raw data and the '
               'channels.tsv file. For MNE-BIDS, the channel names in the '
               'tsv MUST be equal and in the same order as the channels in '
               'the raw data.\n\n'
               '{} channels in tsv file: "{}"\n\n --> {}\n\n'
               '{} channels in raw file: "{}"\n\n --> {}\n\n'
               .format(len(ch_names_tsv), channels_fname, ch_names_tsv,
                       len(ch_names_raw), bids_fname, ch_names_raw)
               )

        # XXX: this could be due to MNE inserting a 'STI 014' channel as the
        # last channel: In that case, we can work. --> Can be removed soon,
        # because MNE will stop the synthesis of stim channels in the near
        # future
        if not (ch_names_raw[-1] == 'STI 014' and
                ch_names_raw[:-1] == ch_names_tsv):
            raise RuntimeError(msg)

    # Now we can do some work.
    # The "type" column is mandatory in BIDS. We can use it to set channel
    # types in the raw data using a mapping between channel types
    channel_type_dict = dict()

    # Get the best mapping we currently have from BIDS to MNE nomenclature
    bids_to_mne_ch_types = _get_ch_type_mapping(fro='bids', to='mne')
    ch_types_json = channels_dict['type']
    for ch_name, ch_type in zip(ch_names_tsv, ch_types_json):

        # Try to map from BIDS nomenclature to MNE, leave channel type
        # untouched if we are uncertain
        updated_ch_type = bids_to_mne_ch_types.get(ch_type, None)
        if updated_ch_type is not None:
            channel_type_dict[ch_name] = updated_ch_type

    # Set the channel types in the raw data according to channels.tsv
    raw.set_channel_types(channel_type_dict)

    # Check whether there is the optional "status" column from which to infer
    # good and bad channels
    if 'status' in channels_dict:
        # find bads from channels.tsv
        bad_bool = [True if chn.lower() == 'bad' else False
                    for chn in channels_dict['status']]
        bads = np.asarray(channels_dict['name'])[bad_bool]

        # merge with bads already present in raw data file (if there are any)
        unique_bads = set(raw.info['bads']).union(set(bads))
        raw.info['bads'] = list(unique_bads)

    return raw

In [ ]:
def _get_ch_type_mapping(fro='mne', to='bids'):
    """Map between BIDS and MNE nomenclatures for channel types.
    Parameters
    ----------
    fro : str
        Mapping from nomenclature of `fro`. Can be 'mne', 'bids'
    to : str
        Mapping to nomenclature of `to`. Can be 'mne', 'bids'
    Returns
    -------
    ch_type_mapping : collections.defaultdict
        Dictionary mapping from one nomenclature of channel types to another.
        If a key is not present, a default value will be returned that depends
        on the `fro` and `to` parameters.
    Notes
    -----
    For the mapping from BIDS to MNE, MEG channel types are ignored for now.
    Furthermore, this is not a one-to-one mapping: Incomplete and partially
    one-to-many/many-to-one.
    """
    if fro == 'mne' and to == 'bids':
        map_chs = dict(eeg='EEG', misc='MISC', stim='TRIG', emg='EMG',
                       ecog='ECOG', seeg='SEEG', eog='EOG', ecg='ECG',
                       # MEG channels
                       meggradaxial='MEGGRADAXIAL', megmag='MEGMAG',
                       megrefgradaxial='MEGREFGRADAXIAL',
                       meggradplanar='MEGGRADPLANAR', megrefmag='MEGREFMAG',
                       )
        default_value = 'OTHER'

    elif fro == 'bids' and to == 'mne':
        map_chs = dict(EEG='eeg', MISC='misc', TRIG='stim', EMG='emg',
                       ECOG='ecog', SEEG='seeg', EOG='eog', ECG='ecg',
                       # No MEG channels for now
                       # Many to one mapping
                       VEOG='eog', HEOG='eog',
                       )
        default_value = 'misc'

    else:
        raise ValueError('Only two types of mappings are currently supported: '
                         'from mne to bids, or from bids to mne. However, '
                         'you specified from "{}" to "{}"'.format(fro, to))

    # Make it a defaultdict to prevent key errors
    ch_type_mapping = defaultdict(lambda: default_value)
    ch_type_mapping.update(map_chs)

    return ch_type_mapping

In [ ]:
"""

# First, make sure that ordering of names in channels.tsv matches the
# ordering of names in the raw data. The "name" column is mandatory in BIDS
ch_names_raw = [ch_name.encode('utf-8') for ch_name in raw.ch_names]
ch_names_tsv = channels_dict['name']
if ch_names_raw != ch_names_tsv:

    msg = ('Channels do not correspond between raw data and the '
           'channels.tsv file. For MNE-BIDS, the channel names in the '
           'tsv MUST be equal and in the same order as the channels in '
           'the raw data.\n\n'
           )

    # XXX: this could be due to MNE inserting a 'STI 014' channel as the
    # last channel: In that case, we can work. --> Can be removed soon,
    # because MNE will stop the synthesis of stim channels in the near
    # future
    if not (ch_names_raw[-1] == 'STI 014' and
            ch_names_raw[:-1] == ch_names_tsv):
        raise RuntimeError(msg)
    else: 
        ch_names_raw = ch_names_raw[:-1]
        
        
"""

In [ ]:
channels_fname = os.path.join(eeg_path,'sub-002_task-FaceRecognition_channels.tsv')
raw = _handle_channels_reading(channels_fname, None, raw)


In [ ]:
import json 

def _read_dig_bids(electrodes_fpath, coordsystem_fpath,
                   raw, kind, verbose):
    """Read MNE-Python formatted DigMontage from BIDS files.
    Handles coordinatesystem.json and electrodes.tsv reading
    to DigMontage.
    Parameters
    ----------
    electrodes_fpath : str
        Filepath of the electrodes.tsv to read.
    coordsystem_fpath : str
        Filepath of the coordsystem.json to read.
    raw : instance of Raw
        The data as MNE-Python Raw object.
    kind : str
        Type of the data as in ALLOWED_KINDS.
    verbose : bool
        Set verbose output to true or false.
    Returns
    -------
    raw : instance of Raw
        The data as MNE-Python Raw object.
    """
    
    space = ''
    
    # read in coordinate information
    with open(coordsystem_fpath, 'r') as fin:
        coordsystem_json = json.load(fin)
    
    print(coordsystem_json)
    coord_frame = coordsystem_json['EEGCoordinateSystem'].lower()
    coord_unit = coordsystem_json['EEGCoordinateUnits']
    coord_frame_desc = coordsystem_json.get('EEGCoordinateDescription',
                                            None)
    
    # only accept captrak
    if coord_frame not in ['captrak','ars']:
        coord_frame = None
    else:
        coord_frame = 'head'
    
# check coordinate units
    if coord_unit not in ['m', 'cm', 'mm']:
        coord_frame = None
    # only set montage if coordinate frame was properly parsed
    if coord_frame is not None:
        print('Read in electrode coordinates and attach to raw')
        raw = _handle_electrodes_reading(electrodes_fpath, coord_frame,
                                         coord_unit, raw, verbose)

    return raw

In [ ]:
# Try to find an associated electrodes.tsv and coordsystem.json
# to get information about the status and type of present channels

electrodes_fname = os.path.join(eeg_path,'sub-002_task-FaceRecognition_electrodes.tsv')
coordsystem_fname = os.path.join(eeg_path,'sub-002_task-FaceRecognition_coordsystem.json')

raw = _read_dig_bids(electrodes_fname, coordsystem_fname,
                             raw, kind='eeg', verbose=1)

In [ ]:
mne.channels.montage.DigMontage()

In [ ]:
00